# Stanley Steering Controller

---

## What Is Stanley Control?

Stanley control is a geometric path tracking algorithm used for steering control in autonomous vehicles. It was developed by Stanford University’s team during the DARPA Grand Challenge.

Stanley Control uses the front axle as the reference point and accounts for:

- Heading error (ψ): the difference between the vehicle’s orientation and the path’s direction.
- Cross-track error (e): the lateral distance from the front axle of the car to the closest point on the path.

The goal is to compute a steering angle δ(t) that aligns the car with the path over time.

---

## Geometric Setup

Given:

- (x, y): rear axle position of the vehicle  
- ψ: Yaw error or heading error = θ - car_yaw
- v: vehicle speed  
- (xf, yf): front axle position = (x + L cos(car_yaw), y + L sin(car_yaw))  
- θ: tangent (heading) of the path at the nearest point  

The control law combines both error terms into the steering command:

$
\delta(t) = k_y \psi(t) + \tan^{-1}\left( \frac{k \cdot e(t)}{v + k_s} \right)
$

Where:

- k is a gain parameter for how strongly the cross-track error is corrected.  
- kₛ is a small softening constant to prevent division by zero when speed is low.  
- δ(t) is clipped between steering angle limits.
- k_y is the weight of yaw error which can be useful for tuning the controller

---

## Your Task

Implement the Stanley steering function using the signature below. You will import this function in your animation simulator notebook.

![Stanley Control Diagram](../assets/stanley_diagram.png)
![Stanley Control Formula](../assets/stanley_formula.png)


In [2]:
import numpy as np
import math

def stanley_steering(x, y, yaw, v, waypoints, k=1.0, ks=1e-2, max_steer=np.radians(30)):
    """
    Stanley steering controller.

    Args:
        x, y     : rear axle position of the car
        yaw      : vehicle heading angle (in radians)
        v        : vehicle speed
        waypoints: Nx2 array of path waypoints
        k        : cross-track gain
        ks       : softening term to prevent div by zero
        max_steer: steering angle limits (in radians)

    Returns:
        steer       : steering angle in radians
        target_idx  : index of the nearest waypoint
    """
    # Step 1: Compute front axle position
    L = 2.5  # assume fixed wheelbase
    fx = x + L * np.cos(yaw)
    fy = y + L * np.sin(yaw)

    # Step 2: Find nearest waypoint
    distance = float('inf')
    nearestx=float(0)
    nearesty=float(0)
    index=0
    target_idx=0
    for x, y in waypoints:
        d = np.sqrt((x - fx)**2 + (y - fy)**2)
        if(d<distance):
            distance=d
            nearestx=x
            nearesty=y
            target_idx=index
            index+=1
        else:
            index+=1
            continue

    # Step 3: Compute heading of path at that point
    headingangle=-1/(math.atan((fy-nearestx)/(fx-nearestx)))

    # Step 4: Compute heading error
    headingerror=headingangle-yaw
    # Step 5: Compute signed cross-track error
    #assuming when waypoint is on right of car then e=+ve and negative otherwise
    vector1=[(nearestx-fx),(nearesty-fy)]#vector whose tail is at front axel and head at nearest x,y
    vector2=[(fx-x),(fy-y)]#vector whose tail is at rear axel and head is at front axel
    if(((vector1[0]*vector2[1])-(vector1[1]*vector2[0]))>0):
        e=distance
    else:
        e=-distance

    # Step 6: Compute steering using Stanley law
    steer = headingerror+math.atan((k*e)/(ks+v))
    if(steer>max_steer):
        steer=max_steer
    if(steer<-max_steer):
        steer=-max_steer
    return steer, target_idx
